In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import PIL
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.optim as optim

# from Modules import ConvBN, PoolConvBN, PoolLinearBN, SharpCosSim2d, SharpCosSimLinear, LReLU

from ConvBN import ConvBN as ConvBN_BiasTrick
from LinearBN import LinearBN

In [2]:
class LReLU(nn.Module):
    def __init__(self):
        super(LReLU, self).__init__()
        self.alpha = nn.Parameter(torch.tensor(5.0)) 
    def forward(self, x):
        return torch.nn.functional.relu(self.alpha*x)

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)) # Normalize with mean 0.5 and std 0.5
])

batch_size= 2000
num_workers=2
pin_memory=True

dataset = torchvision.datasets.FashionMNIST(root='../', train=True, download=True, transform=transform)
train_set, val_set = torch.utils.data.random_split(dataset, [58000, 2000])

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=pin_memory)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=pin_memory)

test_set = torchvision.datasets.FashionMNIST(root='../', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=pin_memory)


In [4]:
if torch.cuda.is_available():
    print("CUDA is available")
else:
    print("CUDA is not available")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

CUDA is available


In [5]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()

        self.conv1_out = 32
        self.conv1_size = 5
        self.conv1_padding = 2


        # self.conv2_out = 16
        # self.conv2_size = 5
        # self.conv2_padding = 0

        self.fc1_out = 512
        self.fc2_out = 10

        self.q = 1e-6
        self.bias_trick_par = torch.nn.Parameter(torch.tensor(0.00005))

        # First Convolutional Block

        self.block1 = ConvBN_BiasTrick(in_channels=1, out_channels=self.conv1_out, kernel_size=self.conv1_size, padding=self.conv1_padding, std = .1)

        size = 28 # 28 - 5 + 2 = 25 + 1 = 26
        self.psize = 3
        # Second Convolutional Block
       
        self.block3 = LinearBN(in_features = self.conv1_out * (size//self.psize) * (size//self.psize), out_features=self.fc1_out, std=.05)
        
        
        self.w2 = nn.Parameter(torch.randn(self.fc1_out, self.fc2_out))
        nn.init.normal_(self.w2, mean=0.0, std=.05)

        self.dropout = nn.Dropout(0.3)
        self.dropout2d = nn.Dropout2d(0.25)

        self.relu = LReLU()




    def forward(self, x):
        
        x = F.max_pool2d(self.relu(self.block1(x)), (self.psize,self.psize), padding=0)
        x = self.dropout2d(x)
        x = x.view(x.size(0), -1)
        
        x = self.relu(self.block3(x))
        # x = self.dropout(x)

        x = x + self.bias_trick_par
        x_norm = x / (x.norm(p=2, dim=1, keepdim=True) + self.q)  # Normalize input x
        w2_norm = self.w2 / (self.w2.norm(p=2, dim=1, keepdim=True) + self.q)  # Normalize weights
        x = torch.matmul(x_norm, w2_norm) # Matrix multiplication 

        # Return raw logits (no softmax here, CrossEntropyLoss handles it)
        return x

    def custom_round(self, n):
        remainder = n % 1000
        base = n - remainder
        if remainder >= 101:
            return base + 1000
        elif remainder <= 100:
            return base

    @torch.no_grad()             # no autograd graph
    def flip_sign_(self, tensor: torch.Tensor, percentage: float) -> torch.Tensor:
        """
        Flip the sign of a random subset of elements *in place*.
    
        Args:
            tensor (torch.Tensor): Any shape, modified in place.
            percentage (float): 0‒1 fraction of elements to flip.
    
        Returns:
            torch.Tensor: The same tensor object (for chaining).
        """
        if percentage <= 0.0:
            return tensor
        if percentage >= 1.0:
            tensor.mul_(-1)
            return tensor                    # all elements flipped
    
        numel = tensor.numel()
        num_to_flip = int(numel * percentage)
        if num_to_flip == 0:
            return tensor
    
        flat = tensor.view(-1)               # view ↔ no copy
        idx = torch.randint(0, numel, (num_to_flip,),
                            device=flat.device)
        flat[idx] *= -1                      # in-place sign change
        return tensor
            

    def init_hdc(self, ratio, seed, flip_perc=None):
        if not isinstance(ratio, (tuple)):
            raise TypeError("ratio must be a tuple of size 3")

        if not isinstance(seed, (tuple)):
            raise TypeError("seed must be a tuple of size 3")
        
        self.block1.init_hdc(ratio = ratio[0], seed = seed[0], flip_perc=flip_perc)
        self.block3.init_hdc(ratio = ratio[1], seed = seed[1], flip_perc=flip_perc)
        
        torch.manual_seed(400) # To change the seed ========================================================
        
        n_last = self.w2.size(0)
        self.nHDC_last = int(self.custom_round(ratio[2] * n_last)) if ratio[2]<1000 else int(ratio[2])
        torch.manual_seed(seed[2])
        self.g = (torch.randn(self.w2.size(0), self.nHDC_last, device=self.w2.device)).to(torch.half)
        self.wg = torch.sign(torch.matmul(self.g.t(), self.w2.to(torch.half)))

        # print(f'Block1: {self.block1.nHDC}, Block2: {self.block3.nHDC}, Classification Layer: {self.nHDC_last}')
        if flip_perc is not None and flip_perc > 0.0:
            self.flip_sign_(self.wg, flip_perc)

    def hdc(self, x):
        x = F.max_pool2d(self.relu(self.block1.hdc(x)), (self.psize, self.psize), padding=0)

        x = x.view(x.size(0), -1)
        x = self.relu(self.block3.hdc(x))

        x = x + self.bias_trick_par
        x = torch.sign(torch.matmul(x.to(torch.half), self.g))

        return x
        
    def classification_layer(self, x):
        x = x @ self.wg
        return x


In [6]:
scales = np.arange(0.0, 0.51, 0.05)
hyperdim = (25000, 25000, 25000)

In [7]:
from tqdm import tqdm
import time
from torch.utils.data import Subset


torch.cuda.empty_cache()
model = Network().to(device)
model.load_state_dict(torch.load('Fashionmnist_GNet_Training.pth', weights_only = True))

model.to(torch.half).to(device)
model.eval()

n_splits = 20
split_size = len(test_set) // n_splits 
accuracies = np.zeros((len(scales), n_splits))
print(len(scales))
for i, perc in enumerate(scales):
    indices = list(range(len(test_set)))
    np.random.seed(42)
    np.random.shuffle(indices)
    pbar = tqdm(range(n_splits))
    for split_idx in pbar:  # Initial desc
        start_idx = split_idx * split_size
        end_idx = start_idx + split_size
        split_indices = indices[start_idx:end_idx]
        split_subset = Subset(test_set, split_indices)
        split_loader = torch.utils.data.DataLoader(split_subset, batch_size=5, shuffle=False,
                                                   num_workers=num_workers, pin_memory=pin_memory)
        torch.manual_seed(split_idx+4)
        random_seeds = tuple(torch.randint(0, 1000, (1,)).item() for _ in range(3))
        torch.cuda.empty_cache()
        
        model.init_hdc(hyperdim, random_seeds, perc)
        correct = 0
        total = 0
    
        with torch.no_grad():
            for images, labels in (split_loader):
                images, labels = images.cuda(non_blocking=True), labels.cuda(non_blocking=True)
                output = model.hdc(images.to(torch.half))
                output = model.classification_layer(output.to(torch.half))
                _, predicted = torch.max(output.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
    
        acc = 100 * correct / total
    
        accuracies[i, split_idx] = acc
        pbar.set_description(f"Avg Accuracy: {np.mean(accuracies[i, :split_idx+1]):.2f}%, Flip Perc: {np.round(perc, 2)}")
    
    print(f'Average Accuracy: {np.mean(accuracies[i]):.2f}%')
        

11


Avg Accuracy: 90.44%, Flip Perc: 0.0: 100%|██████████| 20/20 [03:45<00:00, 11.30s/it]


Average Accuracy: 90.44%


Avg Accuracy: 90.39%, Flip Perc: 0.05: 100%|██████████| 20/20 [03:47<00:00, 11.39s/it]


Average Accuracy: 90.39%


...

Average Accuracy: 87.33%


Avg Accuracy: 84.96%, Flip Perc: 0.45: 100%|██████████| 20/20 [01:59<00:00,  5.97s/it]


Average Accuracy: 84.96%


Avg Accuracy: 79.64%, Flip Perc: 0.5: 100%|██████████| 20/20 [01:58<00:00,  5.91s/it]

Average Accuracy: 79.64%


In [8]:
from scipy.io import savemat
savemat('FashionMNIST_HDCGNet.mat', {'FashionMNIST_HDCGNet':accuracies})